# 🏛️ Karnataka Bhoomi - Owner Search Tool

**Instructions:**
1. Run Cell 1 to set configuration
2. Run Cell 2 to start search
3. Run Cell 3 to close browser and view results

**Features:**
- Checks ALL Hissa numbers for each survey
- Always selects latest Period
- Logs ALL owner records found (not just matches)


In [3]:
# ═══════════════════════════════════════════════════════════════
# CELL 1: CONFIGURATION - CHANGE THESE VALUES
# ═══════════════════════════════════════════════════════════════

OWNER_NAME = "ನಾರಾಯಣಸ್ವಾಮಿ"  # <-- CHANGE THIS TO YOUR SEARCH NAME
OWNER_VARIANTS = [OWNER_NAME, "NARAYANASWAMI"]  # Add English/other variants

LIMIT_DISTRICT = "BANGALORE RURAL"  # District to search
LIMIT_TALUK = "HOSKOTE"              # Taluk to search
LIMIT_HOBLI = "KASABA"               # Hobli to search (None = all hoblis)

MAX_SURVEY_NO = 200  # Max survey number to try per village

print(f"🔍 Owner: {OWNER_NAME}")
print(f"📍 Location: {LIMIT_DISTRICT} > {LIMIT_TALUK} > {LIMIT_HOBLI or 'ALL'}")
print(f"📊 Max surveys: {MAX_SURVEY_NO}")
print("\n✅ Configuration set! Run Cell 2 to start search.")

🔍 Owner: ನಾರಾಯಣಸ್ವಾಮಿ
📍 Location: BANGALORE RURAL > HOSKOTE > KASABA
📊 Max surveys: 200

✅ Configuration set! Run Cell 2 to start search.


In [5]:
# ═══════════════════════════════════════════════════════════════
# CELL 2: RUN SEARCH (Enhanced - All Hissa, Latest Period, Log All)
# ═══════════════════════════════════════════════════════════════

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# CORRECT ELEMENT IDs
IDS = {
    'district': 'ctl00_MainContent_ddlCDistrict',
    'taluk': 'ctl00_MainContent_ddlCTaluk',
    'hobli': 'ctl00_MainContent_ddlCHobli',
    'village': 'ctl00_MainContent_ddlCVillage',
    'survey_no': 'ctl00_MainContent_txtCSurveyNo',
    'surnoc': 'ctl00_MainContent_ddlCSurnocNo',
    'hissa': 'ctl00_MainContent_ddlCHissaNo',
    'period': 'ctl00_MainContent_ddlCPeriod',
    'go_btn': 'ctl00_MainContent_btnCGo',
    'fetch_btn': 'ctl00_MainContent_btnCFetchDetails',
}

SERVICE2_URL = "https://landrecords.karnataka.gov.in/Service2/"
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
ALL_RECORDS_FILE = f"all_records_{timestamp}.csv"
MATCHES_FILE = f"owner_matches_{timestamp}.csv"

def extract_owner_from_page(page_source):
    """Extract owner details from the page"""
    owners = []
    try:
        soup = BeautifulSoup(page_source, 'html.parser')
        for table in soup.find_all('table'):
            text = table.get_text()
            if 'Owner' in text or 'Extent' in text:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    if len(cells) >= 2:
                        cell_texts = [c.get_text(strip=True) for c in cells]
                        # Look for owner data rows (usually has extent pattern)
                        row_text = ' '.join(cell_texts)
                        if re.search(r'\d+\.\d+\.\d+', row_text):  # Extent pattern
                            owners.append({
                                'owner_name': cell_texts[0] if cell_texts else '',
                                'extent': cell_texts[1] if len(cell_texts) > 1 else '',
                                'khatah': cell_texts[2] if len(cell_texts) > 2 else '',
                            })
    except:
        pass
    return owners

# Start browser
print("Starting browser...")
options = Options()
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_records = []  # ALL owner records found
matches = []      # Only records matching search name

try:
    print(f"🔍 Searching for: {OWNER_NAME}")
    print(f"📍 Location: {LIMIT_DISTRICT} > {LIMIT_TALUK} > {LIMIT_HOBLI or 'ALL'}")
    print(f"📁 All records: {ALL_RECORDS_FILE}")
    print(f"📁 Matches: {MATCHES_FILE}")
    print("=" * 60)
    
    driver.get(SERVICE2_URL)
    time.sleep(5)
    
    Select(driver.find_element(By.ID, IDS['district'])).select_by_visible_text(LIMIT_DISTRICT)
    time.sleep(3)
    Select(driver.find_element(By.ID, IDS['taluk'])).select_by_visible_text(LIMIT_TALUK)
    time.sleep(3)
    
    hobli_sel = Select(driver.find_element(By.ID, IDS['hobli']))
    hoblis = [o.text for o in hobli_sel.options if "Select" not in o.text]
    if LIMIT_HOBLI:
        hoblis = [h for h in hoblis if LIMIT_HOBLI in h]
    
    for hobli in hoblis:
        print(f"\n📍 Hobli: {hobli}")
        
        driver.get(SERVICE2_URL)
        time.sleep(3)
        Select(driver.find_element(By.ID, IDS['district'])).select_by_visible_text(LIMIT_DISTRICT)
        time.sleep(2)
        Select(driver.find_element(By.ID, IDS['taluk'])).select_by_visible_text(LIMIT_TALUK)
        time.sleep(2)
        Select(driver.find_element(By.ID, IDS['hobli'])).select_by_visible_text(hobli)
        time.sleep(2)
        
        village_sel = Select(driver.find_element(By.ID, IDS['village']))
        villages = [o.text for o in village_sel.options if "Select" not in o.text]
        
        for village in villages:
            print(f"  📍 Village: {village}")
            empty_count = 0
            
            for survey_no in range(1, MAX_SURVEY_NO + 1):
                try:
                    driver.get(SERVICE2_URL)
                    time.sleep(2)
                    Select(driver.find_element(By.ID, IDS['district'])).select_by_visible_text(LIMIT_DISTRICT)
                    time.sleep(2)
                    Select(driver.find_element(By.ID, IDS['taluk'])).select_by_visible_text(LIMIT_TALUK)
                    time.sleep(2)
                    Select(driver.find_element(By.ID, IDS['hobli'])).select_by_visible_text(hobli)
                    time.sleep(2)
                    Select(driver.find_element(By.ID, IDS['village'])).select_by_visible_text(village)
                    time.sleep(2)
                    
                    driver.find_element(By.ID, IDS['survey_no']).send_keys(str(survey_no))
                    
                    # CLICK GO
                    go_btn = driver.find_element(By.ID, IDS['go_btn'])
                    driver.execute_script("arguments[0].click();", go_btn)
                    time.sleep(8)
                    
                    surnoc_sel = Select(driver.find_element(By.ID, IDS['surnoc']))
                    surnoc_opts = [o.text for o in surnoc_sel.options if "Select" not in o.text]
                    
                    if not surnoc_opts:
                        empty_count += 1
                        if empty_count > 30:
                            print(f"    Skipping after {empty_count} empty")
                            break
                        continue
                    
                    empty_count = 0
                    
                    # Loop through ALL Surnoc options
                    for surnoc in surnoc_opts:
                        surnoc_sel = Select(driver.find_element(By.ID, IDS['surnoc']))
                        surnoc_sel.select_by_visible_text(surnoc)
                        time.sleep(3)
                        
                        hissa_sel = Select(driver.find_element(By.ID, IDS['hissa']))
                        hissa_opts = [o.text for o in hissa_sel.options if "Select" not in o.text]
                        
                        # Loop through ALL Hissa options
                        for hissa in hissa_opts:
                            try:
                                hissa_sel = Select(driver.find_element(By.ID, IDS['hissa']))
                                hissa_sel.select_by_visible_text(hissa)
                                time.sleep(2)
                                
                                # Select LATEST period (first in list is usually latest)
                                period_sel = Select(driver.find_element(By.ID, IDS['period']))
                                period_opts = [o.text for o in period_sel.options if "Select" not in o.text]
                                if period_opts:
                                    # First option is typically the latest
                                    period_sel.select_by_visible_text(period_opts[0])
                                    time.sleep(1)
                                
                                # CLICK FETCH DETAILS
                                fetch_btn = driver.find_element(By.ID, IDS['fetch_btn'])
                                driver.execute_script("arguments[0].click();", fetch_btn)
                                time.sleep(6)
                                
                                page = driver.page_source
                                
                                # Extract ALL owners from page
                                owners = extract_owner_from_page(page)
                                
                                for owner in owners:
                                    record = {
                                        'district': LIMIT_DISTRICT,
                                        'taluk': LIMIT_TALUK,
                                        'hobli': hobli,
                                        'village': village,
                                        'survey_no': survey_no,
                                        'surnoc': surnoc,
                                        'hissa': hissa,
                                        'period': period_opts[0] if period_opts else '',
                                        'owner_name': owner['owner_name'],
                                        'extent': owner['extent'],
                                        'khatah': owner['khatah'],
                                        'timestamp': datetime.now().isoformat()
                                    }
                                    
                                    # Log ALL records
                                    all_records.append(record)
                                    pd.DataFrame(all_records).to_csv(ALL_RECORDS_FILE, index=False)
                                    
                                    # Check if matches search
                                    is_match = any(v in owner['owner_name'] for v in OWNER_VARIANTS)
                                    if is_match:
                                        matches.append(record)
                                        pd.DataFrame(matches).to_csv(MATCHES_FILE, index=False)
                                        print(f"    🎯 MATCH! Sy:{survey_no} H:{hissa} - {owner['owner_name']}")
                                    else:
                                        print(f"    📝 Sy:{survey_no} H:{hissa} - {owner['owner_name'][:20]}")
                                
                                # Re-select for next hissa
                                driver.get(SERVICE2_URL)
                                time.sleep(2)
                                Select(driver.find_element(By.ID, IDS['district'])).select_by_visible_text(LIMIT_DISTRICT)
                                time.sleep(2)
                                Select(driver.find_element(By.ID, IDS['taluk'])).select_by_visible_text(LIMIT_TALUK)
                                time.sleep(2)
                                Select(driver.find_element(By.ID, IDS['hobli'])).select_by_visible_text(hobli)
                                time.sleep(2)
                                Select(driver.find_element(By.ID, IDS['village'])).select_by_visible_text(village)
                                time.sleep(2)
                                driver.find_element(By.ID, IDS['survey_no']).send_keys(str(survey_no))
                                go_btn = driver.find_element(By.ID, IDS['go_btn'])
                                driver.execute_script("arguments[0].click();", go_btn)
                                time.sleep(5)
                                Select(driver.find_element(By.ID, IDS['surnoc'])).select_by_visible_text(surnoc)
                                time.sleep(2)
                                
                            except Exception as e:
                                continue
                
                except Exception as e:
                    empty_count += 1
                    if empty_count > 30:
                        break

    print("\n" + "=" * 60)
    print("✅ SEARCH COMPLETE!")
    print("=" * 60)
    print(f"📊 Total records logged: {len(all_records)}")
    print(f"🎯 Matches found: {len(matches)}")
    print(f"\n📁 All records: {ALL_RECORDS_FILE}")
    print(f"📁 Matches: {MATCHES_FILE}")

except Exception as e:
    print(f"Error: {e}")
    driver.quit()

Starting browser...
🔍 Searching for: ನಾರಾಯಣಸ್ವಾಮಿ
📍 Location: BANGALORE RURAL > HOSKOTE > KASABA
📁 All records: all_records_20251210_123207.csv
📁 Matches: owner_matches_20251210_123207.csv

📍 Hobli: KASABA
  📍 Village: ALAPPANAHALLI
    📝 Sy:1 H:1 - ವೆಂಕಟರಮಣಪ್ಪ
    📝 Sy:2 H:* - D.ಮುನಿಯಪ್ಪ
    🎯 MATCH! Sy:2 H:* - ನಾರಾಯಣಸ್ವಾಮಿ
    📝 Sy:2 H:* - ವೆಂಕಟಪ್ಪ
    📝 Sy:3 H:1 - ಕವಿತಾ ಕೆ
    📝 Sy:3 H:2 - ಮಂಜುನಾಥ ವಿ
    📝 Sy:3 H:3 - ಎ ಮುನಿಯಪ್ಪ
    📝 Sy:3 H:3 - ನರೇಂದ್ರ
    📝 Sy:3 H:4 - ಮುನಿಸಾಮಪ್ಪ ಉ.ಅಬ್ಬಯ್ಯ
    📝 Sy:3 H:6 - ಮುನಿಯಪ್ಪ
    📝 Sy:3 H:7 - ಕವಿತಾ ಕೆ
    📝 Sy:3 H:5 - ಮುನಿಯಪ್ಪ
    📝 Sy:3 H:8 - ವೆಂಕಟಪ್ಪ  ಬಿನ್  ಲೇಟ್
    📝 Sy:3 H:9 - ಪ್ರಕಾಶ್
    📝 Sy:4 H:* - ಖರಾಬ್ ಗೋಮಾಳ
    📝 Sy:4 H:* - ಪುಟ್ಟಣ್ಣ
    📝 Sy:4 H:* - ವೆಂಕಟೇಶ
    📝 Sy:4 H:* - ಚಂದ್ರಶೇಖರ್
    📝 Sy:4 H:* - ವೆಂಕಟರಮಣಪ್ಪ
    📝 Sy:4 H:* - ಬೈರೇಗೌಡ
    📝 Sy:4 H:* - ಪಿ ಕೃಷ್ಣಪ್ಪ
    📝 Sy:5 H:1 - ಮುನಿಯಪ್ಪ
    📝 Sy:5 H:2 - N.ಚಂದ್ರಪ್ಪ
    📝 Sy:5 H:3 - ಆಂಜಿನಮ್ಮ
    📝 Sy:5 H:3 - ವೆಂಕಟರಮಣಪ್ಪ
    📝 Sy:5 H:3 - ಬೈರೇಗೌಡ
    📝 Sy:5 H:3 - ಪಿ ಕೃಷ್ಣಪ್ಪ
    📝 S

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CELL 3: CLOSE BROWSER & VIEW RESULTS
# ═══════════════════════════════════════════════════════════════

driver.quit()
print("✅ Browser closed")

import pandas as pd

print("\n" + "=" * 60)
print("📊 ALL RECORDS FOUND:")
print("=" * 60)
if all_records:
    df_all = pd.DataFrame(all_records)
    display(df_all)
else:
    print("No records found")

print("\n" + "=" * 60)
print("🎯 MATCHING RECORDS:")
print("=" * 60)
if matches:
    df_matches = pd.DataFrame(matches)
    display(df_matches)
else:
    print("No matches found")